# AI Assistant Demo: Simple Log Analyzer

Sample assistant that works with a file.

More on AI Assistants: https://platform.openai.com/docs/assistants

### Setup
Load the API keys and relevant Python libaries. Create an OpenAI client.


In [1]:
import os, time
import requests, json, dotenv
from openai import OpenAI

# Import API key from .env file
openai_api_key = (dotenv.dotenv_values()).get("OPENAI_API_KEY")
my_assistant_id = (dotenv.dotenv_values()).get("ASSISTANT_ID")
my_assistant_name = (dotenv.dotenv_values()).get("ASSISTANT_NAME")

client = OpenAI(api_key=openai_api_key)

### Upload files

In [2]:
"""
file = [None]*1
file[0]=client.files.create(file=open("C:\Cisco\Code\OpenAI\syslog-data.csv", "rb"), purpose="assistants")
"""

my_file_ids = []
for i, file in enumerate(client.files.list()):
    my_file_ids.append(file.id)
    print(f"File object {i}: '{file.id}' {file.filename}")
print(f"File IDs: {my_file_ids}")

File object 0: 'file-mEoxhKcakGGpZx4favKKYJcu' syslog-data.csv
File IDs: ['file-mEoxhKcakGGpZx4favKKYJcu']


### Look for existing assistant by id and name. If it does not exist, create one.

In [3]:
instructions_content = """You are a personal IT support engineer tasked with 
analyzing the attached syslog file formatted in CSV format. 
When asked a question, write and run Python code to answer the question.
"""

assistant_ids = {}
for i, assistant in enumerate(client.beta.assistants.list()):
    assistant_ids[i] = assistant.id
    print(f"Assistant object {i}: '{assistant.id}' {assistant.name}")
    if assistant.id == my_assistant_id:
        print(f"Found assistant by id: {my_assistant_id}")
        break
    elif assistant.name == my_assistant_name:
        my_assistant_id = assistant.id
        print(f"Found assistant by name: {my_assistant_id}")
        break

print("\n------------------------------------------------------------ \n")
print(f"Assistant ID used: {my_assistant_id} \n")

if my_assistant_id is None:
    my_assistant = client.beta.assistants.create(
        model="gpt-4-1106-preview",
        instructions=instructions_content,
        name=my_assistant_name,
        tools=[{"type": "code_interpreter"}], 
        file_ids=my_file_ids,
    )
    my_assistant_id = my_assistant.id
    print(f"New assistant created. Assistant object: {my_assistant} \n")


Assistant object 0: 'asst_LGVeRoE6egTXNeSJSTe6VA1W' Log Analyzer
Found assistant by name: asst_LGVeRoE6egTXNeSJSTe6VA1W

------------------------------------------------------------ 

Assistant ID used: asst_LGVeRoE6egTXNeSJSTe6VA1W 



### Create a Thread

In [4]:
my_thread = client.beta.threads.create()
print(f"This is the thread object: {my_thread} \n")

This is the thread object: Thread(id='thread_xTBtM6DtbvaCmEfyk9YHdYjr', created_at=1705592459, metadata={}, object='thread') 



### Add a Message to a Thread

In [5]:
message_content = """What are important errors and warnings in the syslog file?
What problems do you see described in the syslog file? """

my_thread_message = client.beta.threads.messages.create(
  thread_id=my_thread.id,
  role="user",
  content=message_content,
)
print(f"This is the message object: {my_thread_message} \n")

This is the message object: ThreadMessage(id='msg_V3ShjnYVkHbSnmV1rOUkcoTw', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What are important errors and warnings in the syslog file?\nWhat problems do you see described in the syslog file? '), type='text')], created_at=1705592466, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_xTBtM6DtbvaCmEfyk9YHdYjr') 



### Run the Assistant

In [6]:
my_run = client.beta.threads.runs.create(
  thread_id=my_thread.id,
  assistant_id=my_assistant_id,
  tools=[{"type": "code_interpreter"}, {"type": "retrieval"}]
)
print(f"This is the run object: {my_run} \n")

This is the run object: Run(id='run_vTAeDIrQECSwJIcS9wa65FMX', assistant_id='asst_LGVeRoE6egTXNeSJSTe6VA1W', cancelled_at=None, completed_at=None, created_at=1705592470, expires_at=1705593070, failed_at=None, file_ids=['file-mEoxhKcakGGpZx4favKKYJcu'], instructions='You are a personal IT support engineer tasked with \nanalyzing the attached syslog file formatted in CSV format. \nWhen asked a question, write and run Python code to answer the question.\n', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_xTBtM6DtbvaCmEfyk9YHdYjr', tools=[ToolAssistantToolsCode(type='code_interpreter'), ToolAssistantToolsRetrieval(type='retrieval')]) 



### Periodically retrieve the Run to check on its status to see if it has moved to completed

In [7]:
status = my_run.status
while status in ["queued", "in_progress"]:
    keep_retrieving_run = client.beta.threads.runs.retrieve(
        thread_id=my_thread.id,
        run_id=my_run.id
    )
    status=keep_retrieving_run.status
    time.sleep(0.2)
    print(".", end="")
print(f"\nRun status: {status}")

.....................................................................................................................................................................................................................................................................................................................................................................................
Run status: completed


### Retrieve the Messages added by the Assistant to the Thread

In [8]:
if status == "completed":
    all_messages = client.beta.threads.messages.list(
            thread_id=my_thread.id
    )
    print("\n------------------------------------------------------------ \n")
    print(f"User: {my_thread_message.content[0].text.value}")
    print(f"Assistant: {all_messages.data[0].content[0].text.value}")
else:
    print(f"Exiting with unknown status: {status}")


------------------------------------------------------------ 

User: What are important errors and warnings in the syslog file?
What problems do you see described in the syslog file? 
Assistant: I successfully located the log entry containing the "ERROR" severity level. The message related to this error log begins with a traceback, usually indicative of an exception or error occurring in the system. Here is the content of the error message:

`Dec 19 00:00:35 secpctbmgr1 6d23e148f79a[620]: ERROR  rq.worker    Traceback (most recent call last):`

Unfortunately, the content appears to be cut off after the start of the traceback information. To fully understand the nature of the error and its implications for the system, we would need the complete traceback details which specify the file, line number, and type of error that occurred.

Since we can only see the beginning of the traceback here, I would need to retrieve the subsequent lines of the error message to provide a more precise expl

### Detailed Run Steps of the Assistant

In [9]:
log = []
run_steps = client.beta.threads.runs.steps.list(
        thread_id=my_thread.id, run_id=my_run.id
    )
for i in run_steps.data:
    log.append(
              json.dumps(json.loads(i.step_details.model_dump_json()), indent=2)
    )
print("\n".join(log))

{
  "message_creation": {
    "message_id": "msg_RoBjY8ply8Ul4Vj0uNcaXPqc"
  },
  "type": "message_creation"
}
{
  "tool_calls": [
    {
      "id": "call_MlboXVb3YaJkOmYllOUcbtBc",
      "code_interpreter": {
        "input": "# Re-examine the original warnings and errors dataframe to locate the error message\r\nerror_log_entry = warnings_and_errors[syslog_df.iloc[:,2].str.contains(\"ERROR\")]\r\nerror_log_content = error_log_entry.iloc[0, 2] if not error_log_entry.empty else None\r\nerror_log_content",
        "outputs": [
          {
            "logs": "/tmp/ipykernel_51/2083639690.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.\n  error_log_entry = warnings_and_errors[syslog_df.iloc[:,2].str.contains(\"ERROR\")]\n'Dec 19 00:00:35 secpctbmgr1 6d23e148f79a[620]: ERROR  rq.worker    Traceback (most recent call last):'",
            "type": "logs"
          }
        ]
      },
      "type": "code_interpreter"
    }
  ],
  "type": "tool_calls"
}
{
  "